In [8]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'Desc-Title'
device = 'cuda'

In [9]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [10]:
tokenize('$100')

['$', '100']

In [11]:
def stem(word):
    return stemmer.stem(word.lower())

In [12]:
stem('organic')

'organ'

In [13]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [14]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [15]:
data = pd.read_csv('./data.csv')

In [16]:
data

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...
...,...,...,...,...,...,...
332,Daniel Simmons,3.4K,8,https://itnext.io/you-can-build-a-neural-netwo...,You can build a neural network in JavaScript e...,Click here to share this article on LinkedIn »...
333,Eugenio Culurciello,2.8K,13,https://towardsdatascience.com/artificial-inte...,"Artificial Intelligence, AI in 2018 and beyond...",These are my opinions on where deep neural net...
334,Devin Soni,5.8K,4,https://towardsdatascience.com/spiking-neural-...,"Spiking Neural Networks, the Next Generation o...",Everyone who has been remotely tuned in to rec...
335,Carlos E. Perez,3.9K,7,https://medium.com/intuitionmachine/neurons-ar...,Surprise! Neurons are Now More Complex than We...,One of the biggest misconceptions around is th...


In [17]:
X = data['text']
y = data['title']
X_words = []
y_words = []
data = []

In [18]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    y_batch = tokenize(y_batch)
    new_X = []
    new_y = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    for yb in y_batch:
        new_y.append(stem(yb))
    data.append([new_X,new_y])
    X_words.extend(new_X)
    y_words.extend(new_y)

337it [00:10, 31.84it/s]


In [19]:
X = []
y = []

In [20]:
for desc,title in tqdm(data):
    X.append(bag_of_words(desc,X_words))
    y.append(bag_of_words(title,y_words))

100%|█████████████████████████████████████████| 337/337 [24:29<00:00,  4.36s/it]


In [21]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [26]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.iters = 10
        self.activation = ReLU()
        self.linear1 = Linear(len(X_words),512)
        self.linear2 = Linear(512,512)
        self.linear2bn = BatchNorm1d(512)
        self.output = Linear(512,len(y_words))
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.output(preds)
        return preds

In [28]:
model = Model()
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [ ]:
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    model.train()
wandb.finish()
torch.cuda.empty_cache()

In [25]:
# torch.save(model,'model.pt')
# torch.save(model,'model.pth')
# torch.save(model.state_dict(),'model-sd.pt')
# torch.save(model.state_dict(),'model-sd.pth')
torch.save(X_words,'X_words.pt')
torch.save(y_words,'y_words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
# torch.save(labels,'labels.pt')
# torch.save(labels,'labels.pth')
# torch.save(idx,'idx.pt')
# torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')